In [1]:
import os
from dotenv import load_dotenv
import json
import re

load_dotenv()

True

In [2]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [4]:
category_file = "category.json"

In [5]:
with open(category_file, 'r', encoding='utf-8') as f:
    category_data = json.load(f)

In [6]:
category_data.get('Boards', {}).keys()

dict_keys(['CBSE', 'ICSE', 'IB', 'CISCE', 'CIE', 'NIOS', 'UP Board', 'JKBOSE', 'RBSE', 'HPBOSE', 'MPBSE', 'CGBSE', 'PSEB', 'BSEB', 'GSEB', 'MSBSHSE', 'BIEAP', 'BSEAP', 'WBBSE'])

In [7]:
import json
import os
import re
class EducationContentFetcher:
    def __init__(self, category_file='category.json', topics_file='topics.json', data_folder='data'):
        """Initialize the content fetcher with JSON files and data folder."""
        self.data_folder = data_folder
        
        # Load JSON files
        with open(category_file, 'r', encoding='utf-8') as f:
            self.category_data = json.load(f)
        
        with open(topics_file, 'r', encoding='utf-8') as f:
            self.topics_data = json.load(f)
    
    def get_boards(self):
        """Get list of available boards."""
        return list(self.category_data.get('Boards', {}).keys())
    
    def get_classes(self, board):
        """Get list of available classes for a board."""
        return list(self.category_data['Boards'].get(board, {}).get('Classes', {}).keys())
    
    def get_subjects(self, board, class_num):
        """Get list of available subjects for a board and class."""
        return list(self.category_data['Boards'].get(board, {})
                   .get('Classes', {}).get(str(class_num), {})
                   .get('Subjects', {}).keys())
    
    def get_topics(self, board, class_num, subject):
        """Get list of available topics for a board, class, and subject."""
        topics = self.topics_data.get('Boards', {}).get(board, {}).get('Classes', {}).get(str(class_num), {}).get('Subjects', {}).get(subject, {})
        return {num: name for num, name in topics.items()}
    
    def get_books(self, board, class_num, subject):
        """Get list of books for a board, class, and subject."""
        books = self.category_data['Boards'].get(board, {}).get('Classes', {}).get(str(class_num), {}).get('Subjects', {}).get(subject, {}).get('Books', {})
        return list(books.values())
    
    def extract_topic_from_book(self, book_content, topic_num):
        """
        Extract topic ONLY when chapter is marked as:
            1. ##x##
            2. UNIT-x / Unit-x / unit-x  (dash required)
        """

        topic_num = str(topic_num).strip()

        # Stop when next UNIT-x or ##x##
        next_marker = rf'(?=UNIT\s*-\s*\d+|##\s*\d+\s*##|\Z)'

        patterns = [
            # Pattern 1: ##x##
            rf'##\s*{re.escape(topic_num)}\s*##\s*(.*?){next_marker}',

            # Pattern 2: UNIT-x (dash required)
            rf'UNIT\s*-\s*{re.escape(topic_num)}\s*(.*?){next_marker}',
        ]

        flags = re.IGNORECASE | re.DOTALL

        for pat in patterns:
            match = re.search(pat, book_content, flags)
            if match:
                return match.group(1).strip()

        return None


    
    def fetch_content(self, board, class_num, subject, topic_num):
        """
        Fetch the topic content from the appropriate book.
        
        Returns:
            dict: Contains 'book_name', 'book_id', 'topic_name', 'content', and 'status'
        """
        result = {
            'board': board,
            'class': class_num,
            'subject': subject,
            'topic_num': topic_num,
            'book_name': None,
            'book_id': None,
            'topic_name': None,
            'content': None,
            'status': 'error',
            'message': ''
        }
        
        # Get topic name (optional - may not be in topics.json)
        topics = self.get_topics(board, class_num, subject)
        if topics and str(topic_num) in topics:
            result['topic_name'] = topics[str(topic_num)]
        else:
            result['topic_name'] = f"Topic {topic_num}"
        
        # Get books for this subject
        books = self.get_books(board, class_num, subject)
        if not books:
            result['message'] = "No books found for this subject in category.json"
            return result
        
        # Debug: Show which books we're checking
        print(f"\nSearching in {len(books)} book(s)...")
        
        # Try each book until we find the topic
        books_checked = []
        for book in books:
            book_id = book.get('book_id')
            book_name = book.get('Name')
            book_path = os.path.join(self.data_folder, f"{book_id}.txt")
            
            books_checked.append(book_id)
            print(f"  Checking: {book_name} (ID: {book_id})")
            
            if not os.path.exists(book_path):
                print(f"    ❌ File not found: {book_path}")
                continue
            
            try:
                print(f"    ✓ File found, searching for topic ###{topic_num}##...")
                with open(book_path, 'r', encoding='utf-8') as f:
                    book_content = f.read()
                
                # Extract the specific topic
                topic_content = self.extract_topic_from_book(book_content, topic_num)
                
                if topic_content:
                    print(f"    ✓ Topic found!")
                    result['book_name'] = book_name
                    result['book_id'] = book_id
                    result['content'] = topic_content
                    result['status'] = 'success'
                    result['message'] = 'Topic found successfully'
                    return result
                else:
                    print(f"    ❌ Topic ###{topic_num}## not found in this book")
            
            except Exception as e:
                print(f"    ❌ Error reading file: {e}")
                continue
        
        result['message'] = f"Topic {topic_num} not found in any available books. Checked: {', '.join(books_checked)}"
        return result
    
    def display_result(self, result):
        """Display the fetched content in a formatted way."""
        print("\n" + "="*70)
        print(f"Board: {result['board']}")
        print(f"Class: {result['class']}")
        print(f"Subject: {result['subject']}")
        print(f"Topic: {result['topic_name']} (Topic #{result['topic_num']})")
        print("="*70)
        
        if result['status'] == 'success':
            print(f"\nBook: {result['book_name']} (ID: {result['book_id']})")
            print("\nContent:")
            print("-"*70)
            print(result['content'])
            print("-"*70)
            return(result['content'])
        else:
            print(f"\nError: {result['message']}")
        print()

In [79]:
def main():
    """Fetch content with hardcoded values."""
    try:
        # HARDCODED VALUES - Change these as needed
        selected_board = "CBSE"
        selected_class = "11"
        selected_subject = "Physics"
        selected_topic = "4" 
        
        # Initialize fetcher
        fetcher = EducationContentFetcher()
        
        print("=" * 70)
        print("Education Content Fetcher".center(70))
        print("=" * 70)
        print(f"\nFetching content for:")
        print(f"  Board: {selected_board}")
        print(f"  Class: {selected_class}")
        print(f"  Subject: {selected_subject}")
        print(f"  Topic: {selected_topic}")
        print("\n" + "=" * 70)
        
        # Fetch and display content
        result = fetcher.fetch_content(
            selected_board, 
            selected_class, 
            selected_subject, 
            selected_topic
        )
        return fetcher.display_result(result)
        
    except FileNotFoundError as e:
        print(f"\nError: Required file not found - {e}")
    except KeyError as e:
        print(f"\nError: Invalid selection or data structure - {e}")
    except Exception as e:
        print(f"\nError: {e}")


if __name__ == "__main__":
    res = main()

                      Education Content Fetcher                       

Fetching content for:
  Board: CBSE
  Class: 11
  Subject: Physics
  Topic: 4


Searching in 2 book(s)...
  Checking: Physics Part-I (NCERT) (ID: C11PHY0PHYS0NCERT)
    ✓ File found, searching for topic ###4##...
    ✓ Topic found!

Board: CBSE
Class: 11
Subject: Physics
Topic: Topic 4 (Topic #4)

Book: Physics Part-I (NCERT) (ID: C11PHY0PHYS0NCERT)

Content:
----------------------------------------------------------------------
LAWS OF MOTION

4.1  INTRODUCTION

In the preceding Chapter, our concern was to describe the
motion of a particle in space quantitatively. We saw that

4.1 Introduction uniform motion needs the concept of velocity alone whereas
4.2 Aristotle’s fallacy non-uniform motion requires the concept of acceleration in
4.3 The law of inertia addition.  So far, we have not asked the question as to what

4.4 Newton’s first law of motion governs the motion of bodies. In this chapter, we turn to this

4.

In [80]:
res

'LAWS OF MOTION\n\n4.1  INTRODUCTION\n\nIn the preceding Chapter, our concern was to describe the\nmotion of a particle in space quantitatively. We saw that\n\n4.1 Introduction uniform motion needs the concept of velocity alone whereas\n4.2 Aristotle’s fallacy non-uniform motion requires the concept of acceleration in\n4.3 The law of inertia addition.  So far, we have not asked the question as to what\n\n4.4 Newton’s first law of motion governs the motion of bodies. In this chapter, we turn to this\n\n4.5 basic question.\nNewton’s second law of\nmotion Let us first guess the answer based on our common\n\n4.6 experience. To move a football at rest, someone must kick it.\nNewton’s third law of motion\n\nTo throw a stone upwards, one has to  give it an upward\n4.7 Conservation of momentum\n\npush.  A breeze causes the branches of a tree to swing; a\n4.8 Equilibrium of a particle strong wind can even move heavy objects. A boat moves in a\n4.9 Common forces in mechanics flowing river withou

In [81]:
from langchain_core.documents import Document

docs = [Document(page_content=res)]
docs

[Document(metadata={}, page_content='LAWS OF MOTION\n\n4.1  INTRODUCTION\n\nIn the preceding Chapter, our concern was to describe the\nmotion of a particle in space quantitatively. We saw that\n\n4.1 Introduction uniform motion needs the concept of velocity alone whereas\n4.2 Aristotle’s fallacy non-uniform motion requires the concept of acceleration in\n4.3 The law of inertia addition.  So far, we have not asked the question as to what\n\n4.4 Newton’s first law of motion governs the motion of bodies. In this chapter, we turn to this\n\n4.5 basic question.\nNewton’s second law of\nmotion Let us first guess the answer based on our common\n\n4.6 experience. To move a football at rest, someone must kick it.\nNewton’s third law of motion\n\nTo throw a stone upwards, one has to  give it an upward\n4.7 Conservation of momentum\n\npush.  A breeze causes the branches of a tree to swing; a\n4.8 Equilibrium of a particle strong wind can even move heavy objects. A boat moves in a\n4.9 Common forc

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


d:\Drema Ai\vev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 200,
)

split = text_splitter.split_documents(documents=docs)
print(len(split))
vector_store = FAISS.from_documents(
    documents=split,
    embedding=embeddings,
)

retriever = vector_store.as_retriever(
    search_kwargs = {
        "k":5
    }
)

128


In [83]:
response = retriever.invoke("Motion")
response

[Document(id='9eff5504-e6c6-4558-881c-41d0c742fb5e', metadata={}, page_content='9. The different terms like ‘friction’, ‘normal reaction’ ‘tension’, ‘air resistance’,\n‘viscous drag’, ‘thrust’, ‘buoyancy’, ‘weight’, ‘centripetal force’ all stand for ‘force’\nin different contexts.  For clarity, every force and its equivalent terms\nencountered in mechanics should be reduced to the phrase ‘force on A by B’.\n\n10. For applying the second law of motion, there is no conceptual distinction between\ninanimate and animate objects.  An animate object such as a human also\nrequires an external  force to accelerate.  For example, without the external\nforce of friction, we cannot walk on the ground.'),
 Document(id='60684775-b784-4c94-a95f-4aab6944ddb3', metadata={}, page_content='11. The objective concept of force in physics should not be confused with the\nsubjective concept of the ‘feeling of force’.  On a merry-go-around, all parts of\nour body are subject to an  inward force,  but we have 

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
    streaming=True,
    temperature=0.7,
)

d:\Drema Ai\vev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [85]:
llm.invoke(f"Can you answer using {response} that. What is Motion and it's laws? The answer should be precise and to the point no extra addition from you side. Read the whole context and give me the answer.")

AIMessage(content="According to the provided context, Motion is described as follows:\n\n- Every body continues to be in its state of rest or of uniform motion in a straight line unless compelled by some external force to act otherwise. (Galileo's first law of motion)\n\n- A force is required to keep a body in motion. (Aristotelian law of motion)\n\n- An object moving on a frictionless horizontal plane must neither have acceleration nor retardation, i.e. it should move with constant velocity. (Aristotelian view on motion)\n\n- The second law of motion states that force F at a point in space (location of the particle) at a certain instant is related to a at that point at that instant, determining acceleration. (F = ma)\n\nThe laws of motion are not explicitly stated in the provided context, but they can be inferred as:\n\n- First Law of Motion: A body at rest or in uniform motion will remain so unless an external force is applied.\n- Second Law of Motion: Force is equal to mass times ac

In [86]:
def _reformulate_query(question):
    """Reformulate the query for better retrieval precision."""
    contextualize_prompt = ChatPromptTemplate.from_messages([
        ("system", 
         "You are an expert at reformulating search queries for educational content retrieval.\n\n"
         "Your task:\n"
         "1. Analyze the user's question carefully\n"
         "2. Extract key concepts, entities, and educational terms\n"
         "3. Expand abbreviations and clarify ambiguous terms\n"
         "4. Reformulate into a clear, specific search query that will retrieve the most relevant content\n"
         "5. Keep the reformulated query concise (1-2 sentences max)\n"
         "6. If the question is already clear and specific, return it as-is\n\n"
         "Examples:\n"
         "User: 'What's this about?'\n"
         "Reformulated: 'What is the main topic and key concepts covered in this chapter?'\n\n"
         "User: 'Explain photosynthesis'\n"
         "Reformulated: 'Explain the process of photosynthesis, including its stages, inputs, and outputs'\n\n"
         "Do NOT answer the question. Only return the reformulated search query."
        ),
        ("human", "{question}")
    ])
        
    chain = contextualize_prompt | llm
    response = chain.invoke({"question": question})
    reformulated = response.content.strip()
        
    print(f"📝 Reformulated query: {reformulated}")
    return reformulated

In [87]:
def ask(question):
    reformulated_query = _reformulate_query(question)

    docs = retriever.invoke(reformulated_query)

    # FIX 1: Correct doc joining
    context = "\n\n".join([doc.page_content for doc in docs])
    print(context)

    system_prompt = (
        "You are an expert educational AI assistant specialized in explaining academic content.\n\n"
        "## Your Core Principles:\n"
        "1. Accuracy First — use ONLY the provided context.\n"
        "2. No Fabrication — if missing, say: 'This information is not available in the given chapter.'\n"
        "3. Educational Clarity — explain simply and precisely.\n"
        "4. Structured Responses — but keep answers short.\n"
        "5. Read the whole context and answer strictly based on it.\n\n"
        "## Context:\n"
        f"{context}\n\n"
        "Now answer the student's question using ONLY the context above. " 
        "The answer should be precise and to the point no extra addition from you side. Read the whole context and give me the answer."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{question}")
    ])

    chain = prompt | llm
    response = chain.invoke({"question": question})
    return response.content


In [88]:
rag = ask("Can you summarize this chapter")

📝 Reformulated query: What is a summary of this chapter, and what key points or concepts are covered within it?
statement is : “Since the book is observed to be at are thrown backward.  As soon as that happens,
rest, the net external force on it must be zero, however, the muscular forces on the rest of the
according to the first law. This implies that the body (by the feet) come into play to move the body
normal force R  must be equal and opposite to the along with the bus. A similar thing happens
weight W ”. when the bus suddenly stops.  Our feet stop due

means that, the particle is either at rest or in
uniform motion.

If two forces F  and F , act on a particle,
1 2

equilibrium requires

F = − F       (4.10)
1   2

i.e. the two forces on the particle  must be equal
and opposite. Equilibrium under three
concurrent forces F , F   and F  requires that

1 2 3 (a) (b) (c)
the vector sum of the three forces is zero. Fig. 4.8

F   + F   + F   =  0                                  (4.11)
1

In [89]:
print(rag)

This chapter discusses the laws of motion. Here's a summary based on the provided context:

1. **First Law (Equilibrium)**: A particle is in equilibrium if the net external force on it is zero. For equilibrium under multiple forces, the vector sum of the forces must be zero (F1 + F2 + F3 = 0).
2. **Second Law (Force and Acceleration)**: The total external force on a system (F) is equal to the mass of the system (m) multiplied by its acceleration (a) (F = ma). The acceleration of a system is determined by the force at a given instant, not by the history of motion.
3. **Impulse and Momentum**: Impulse is the product of force and time, which equals the change in momentum. The notion of impulse is useful when a large force acts for a short time to produce a measurable change in momentum.
4. **Newton's Third Law of Motion**: To every action, there is always an equal and opposite reaction.

This chapter emphasizes the concepts of equilibrium, force, acceleration, impulse, and Newton's third 

In [90]:
def ask(question):

    reformulated_query = _reformulate_query(question)


    docs = retriever.invoke(reformulated_query)
    print(docs)
    context = "\n\n".join([doc.page_content for doc in docs])
    # Create prompt
    system_prompt = (
        "You are an expert educational AI assistant specialized in explaining academic content.\n\n"
            "## Your Core Principles:\n"
            "1. **Accuracy First**: Base your answers EXCLUSIVELY on the provided context\n"
            "2. **No Fabrication**: If information isn't in the context, clearly state: "
            "'This information is not available in the current chapter content.'\n"
            "3. **Educational Clarity**: Explain concepts in a clear, student-friendly manner\n"
            "4. **Structured Responses**: Use examples, analogies, or step-by-step explanations when helpful\n\n"
            "5. Understand the whole context and answer the user query."
            "## Response Guidelines:\n"
            "- For definitions: Provide clear, concise explanations\n"
            "- For processes: Break down into logical steps\n"
            "- For concepts: Explain with relevant examples from the context\n"
            "- For comparisons: Highlight key similarities and differences\n"
            "- For 'why' questions: Explain reasoning and underlying principles\n\n"
            "## What to Avoid:\n"
            "- Do NOT add external knowledge not present in the context\n"
            "- Do NOT make assumptions beyond what's stated\n"
            "- Do NOT provide opinions or subjective interpretations\n"
            "- Do NOT copy text verbatim—paraphrase and synthesize\n\n"
            "## Context from Chapter:\n"
            f"{context}\n\n"
            "Now answer the student's question using ONLY the information above"
    )
        
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{question}")
    ])

    chain = prompt | llm
    response = chain.invoke({"question":question})
    return response.content

In [91]:
rag = ask("What is Motion and its laws?")

📝 Reformulated query: What are the fundamental principles and laws of motion as described in classical mechanics and physics?
[Document(id='598acbbf-8a0a-46ad-95e3-86ba9fb9dc9f', metadata={}, page_content='Galileo thus, arrived at a new insight on lated as the first law of motion:\nmotion that had eluded Aristotle and those who Every body continues to be in its state\nfollowed him.  The state of rest and the state of of rest or of uniform motion in a straight\nuniform linear motion (motion with constant line unless compelled by some external\nvelocity) are equivalent. In both cases, there is force to act otherwise.\n\nIdeas on Motion in Ancient Indian Science'), Document(id='d446a0be-5236-4a70-92e6-6a75d3fb3d7b', metadata={}, page_content='(iii) Motion on a horizontal plane  is an interme-\nsomething external is required to keep it moving.\n\ndiate situation.  Galileo concluded that an object\nAccording to this view, for example, an arrow\n\nmoving on a frictionless horizontal plane mu

In [92]:
print(rag)

Motion refers to the change in position of an object over time. According to the context, there are three main laws governing the motion of bodies:

**Aristotle's Law of Motion (now known to be wrong)**:
An external force is required to keep a body in motion.

**Galileo's Law of Motion (also known as the Law of Inertia)**:
Every body continues to be in its state of rest or of uniform motion in a straight line (with constant velocity) unless compelled by some external force to act otherwise.

**Newton's Law of Motion (as stated by Galileo, but later rephrased by Newton)**:
Everybody continues to be in its state of rest or of uniform motion in a straight line, unless compelled by some external force to act otherwise.

In simple terms, the First Law (Law of Inertia) is: "If external force on a body is zero, its acceleration is zero." This means that an object at rest will remain at rest, and an object in motion will continue to move with a constant velocity, unless acted upon by an extern

In [93]:
def generate_mcqs(num_questions, difficulty_level, topic):
    """
        Generate MCQs based on the chapter content.
        
        Args:
            num_questions (int): Number of MCQs to generate
            difficulty_level (str): 'easy', 'medium', or 'hard'
        
        Returns:
            list: List of MCQ dictionaries
    """
    # Validate difficulty level
    if difficulty_level.lower() not in ['easy', 'medium', 'hard']:
        return "Invalid difficulty level. Choose from: easy, medium, hard"
    # Retrieve relevant content from the chapter
    query = f"Generate questions covering this {topic} from the chapter"

    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Create MCQ generation prompt
    mcq_prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are an expert educational assessment creator specializing in Multiple Choice Questions (MCQs).\n\n"
             "## Your Task:\n"
             f"Generate EXACTLY {num_questions} Multiple Choice Questions at **{difficulty_level.upper()} difficulty level** "
             "based STRICTLY on the provided chapter content.\n\n"
             "## Difficulty Level Guidelines:\n"
             "### EASY:\n"
             "- Test direct recall and basic understanding\n"
             "- Questions about definitions, simple facts, and key terms\n"
             "- Straightforward language\n"
             "- Example: 'What is photosynthesis?'\n\n"
             "### MEDIUM:\n"
             "- Test comprehension and application\n"
             "- Questions requiring understanding of relationships and processes\n"
             "- May involve simple analysis or comparison\n"
             "- Example: 'How does photosynthesis differ from respiration?'\n\n"
             "### HARD:\n"
             "- Test analysis, synthesis, and evaluation\n"
             "- Questions requiring critical thinking and deeper understanding\n"
             "- May involve problem-solving or application to new scenarios\n"
             "- And try to implement such that the question will cover 2-3 topics."
             "- In case of maths, physics and chemistry set question such that it uses concept of 2-3 topics and "
                "set good quallity questions."
             "- Example: 'What would happen to the rate of photosynthesis if CO2 levels decreased?'\n\n"
             "## MCQ Format Requirements:\n"
             "1. Each question must have EXACTLY 4 options (A, B, C, D)\n"
             "2. ONLY ONE option should be correct\n"
             "3. Distractors (wrong options) should be plausible but clearly incorrect\n"
             "4. Avoid 'All of the above' or 'None of the above' options\n"
             "5. Options should be roughly similar in length\n"
             "6. Questions must be clear, unambiguous, and grammatically correct\n\n"
             "7. Question can be numericals as well where it is needed."
             "## Output Format:\n"
             "Return ONLY valid JSON (no markdown, no code blocks) in this exact structure:\n"
             "{{\n"
             '  "mcqs": [\n'
             "    {{\n"
             '      "question": "Question text here?",\n'
             '      "options": {{\n'
             '        "A": "Option A text",\n'
             '        "B": "Option B text",\n'
             '        "C": "Option C text",\n'
             '        "D": "Option D text"\n'
             "      }},\n"
             '      "correct_answer": "A",\n'
             '      "explanation": "Brief explanation why this answer is correct"\n'
             "    }}\n"
             "  ]\n"
             "}}\n\n"
             "## Important Rules:\n"
             "- Base ALL questions on the provided context only\n"
             "- Do NOT create questions about information not in the context\n"
             "- Ensure questions are diverse and cover different aspects of the chapter\n"
             "- Maintain consistent difficulty level across all questions\n"
             f"- Generate EXACTLY {num_questions} questions, no more, no less\n\n"
             "## Chapter Content:\n"
             f"{context}"
            ),
            ("human", f"Generate {num_questions} {difficulty_level} level MCQs from the chapter content provided.")
    ])

    # Generate MCQs
    chain = mcq_prompt | llm
    response = chain.invoke({})

    try:
            # Parse JSON response
        import json
        # Clean response if it has markdown code blocks
        response_text = response.content.strip()
        if response_text.startswith("```"):
            response_text = response_text.split("```")[1]
            if response_text.startswith("json"):
                response_text = response_text[4:]
            
        mcqs_data = json.loads(response_text)
        return mcqs_data['mcqs']
    except Exception as e:
        print(f"Error parsing MCQs: {e}")
        print(f"Raw response: {response.content}")
        return None

In [94]:
@staticmethod
def display_mcqs(mcqs):
    """Display MCQs in a formatted way."""
    if not mcqs:
        print("No MCQs to display")
        return
        
    print("\n" + "=" * 70)
    print("GENERATED MCQs".center(70))
    print("=" * 70)
        
    for i, mcq in enumerate(mcqs, 1):
        print(f"\nQ{i}. {mcq['question']}")
        for option, text in mcq['options'].items():
            print(f"   {option}. {text}")
        print(f"\n   ✓ Correct Answer: {mcq['correct_answer']}")
        print(f"   💡 Explanation: {mcq['explanation']}")
        print("-" * 70)

In [95]:
topics = ["Projectile Motion", "Laws of motion"]

In [96]:
num_question = 5
difficulty_level = "hard"

In [97]:
mcqs = []
for topic in topics:
    mcqs.append(generate_mcqs(num_question, difficulty_level, topic=topic))

In [98]:
for mcq in mcqs:
    display_mcqs(mcqs=mcq)


                            GENERATED MCQs                            

Q1. A shell of mass 0.020 kg is fired by a gun of mass 100 kg. If the muzzle speed of the shell is 80 m s-1, what is the recoil speed of the gun?
   A. 4.8 m s-1
   B. 8 m s-1
   C. 16 m s-1
   D. 32 m s-1

   ✓ Correct Answer: A
   💡 Explanation: Using conservation of momentum, the recoil speed of the gun can be calculated as v_g = m_s * v_s / m_g, where m_s is the mass of the shell, v_s is the muzzle speed of the shell, and m_g is the mass of the gun.
----------------------------------------------------------------------

Q2. A body of mass 0.40 kg moving initially with a constant speed of 10 m s-1 to the north is subject to a constant force of 8.0 N directed towards the south for 30 s. What is the acceleration of the body?
   A. 0.25 m s-2
   B. 0.50 m s-2
   C. 0.67 m s-2
   D. 1.25 m s-2

   ✓ Correct Answer: C
   💡 Explanation: Using Newton's second law, F = m * a, the acceleration of the body can be calcula

In [4]:
llm.invoke("Who are u?")

d:\Drema Ai\vev\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


AIMessage(content="I'm Kimi, your AI friend and assistant from Moonshot AI. I can help you with answers, tasks, or just chat—whatever you need!", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_05df423bab', 'service_tier': 'on_demand', 'model_provider': 'groq'}, id='lc_run--7c45b548-36a7-49e6-84e2-5f516278bbcb', usage_metadata={'input_tokens': 30, 'output_tokens': 32, 'total_tokens': 62})

In [5]:
print(llm.invoke("Write a cpp code for binary search").content)

Here’s a clean, reusable C++ implementation of binary search that works on any sorted container with random-access iterators (e.g., `std::vector`, `std::array`, built-in arrays).  
It returns the index of the key if found, or `-1` if not.

```cpp
#include <iostream>
#include <vector>

// ------------------------------------------------------------------
// Generic binary search
// Works on any random-access iterator range [first, last)
// Returns: zero-based index of key if found, else -1
// ------------------------------------------------------------------
template <typename RandomIt, typename T>
long long binarySearch(RandomIt first, RandomIt last, const T& key)
{
    long long low  = 0;
    long long high = static_cast<long long>(last - first) - 1;

    while (low <= high)
    {
        const long long mid = low + (high - low) / 2;
        if (*(first + mid) == key)
            return mid;
        else if (*(first + mid) < key)
            low = mid + 1;
        else
            hig